In [2]:
import csv

# create a table to search for null values

import pandas as pd
from pathlib import Path

folder = Path("Data")
files = sorted(folder.glob("*.csv"))

rows = []
for f in files:
    df = pd.read_csv(f)
    s = (df.isna().mean() * 100).round(2)
    s.name = f.name
    rows.append(s)

result = pd.DataFrame(rows).fillna(0).sort_index(axis=1)  # columns = all columns seen


print(result)
# result.to_csv("null_percent_table.csv", index=True)

                  Air Pollutant  Air Pollutant Description  \
2023_NO2_IT.csv             0.0                        0.0   
2023_O3_IT.csv              0.0                        0.0   
2023_PM10_IT.csv            0.0                        0.0   
2023_PM25_IT.csv            0.0                        0.0   
2024_NO2_IT.csv             0.0                        0.0   
2024_O3_IT.csv              0.0                        0.0   
2024_PM10_IT.csv            0.0                        0.0   
2024_PM25_IT.csv            0.0                        0.0   

                  Air Pollution Level  Air Quality Network  \
2023_NO2_IT.csv                  0.00                  0.0   
2023_O3_IT.csv                   0.28                  0.0   
2023_PM10_IT.csv                 0.00                  0.0   
2023_PM25_IT.csv                 0.00                  0.0   
2024_NO2_IT.csv                  0.16                  0.0   
2024_O3_IT.csv                   0.43                  0.0   
2024_PM